In [3]:
#!/usr/bin/env python
# coding: utf-8

import sqlite3
import pandas as pd
import json
import os

# Create a connection to a new SQLite database in memory
conn = sqlite3.connect(":memory:")

# Read CSV files into pandas DataFrames
account_df = pd.read_csv("data/account_tab.csv")
transaction_df = pd.read_csv("data/transaction_tab.csv")


# Ensure payload column is valid JSON
def validate_json(payload):
    try:
        return json.dumps(json.loads(payload))
    except:
        return json.dumps({})


transaction_df["payload"] = transaction_df["payload"].apply(validate_json)

# Write the DataFrames to SQLite
account_df.to_sql("account_tab", conn, index=False)
transaction_df.to_sql("transaction_tab", conn, index=False)


# Read and execute SQL queries
def execute_query(file_name):
    with open(f"sql/{file_name}", "r") as file:
        sql_query = file.read()
    return pd.read_sql_query(sql_query, conn)


# Execute queries and save results
result_adoption = execute_query("customer_adoption.sql")
result_usage = execute_query("usage_count.sql")

# Display the results
print("Monthly Customer Adoption:")
print(result_adoption)
print("\nMonthly Usage Count:")
print(result_usage)

# Save results to CSV
result_adoption.to_csv("outputs/monthly_customer_adoption.csv", index=False)
result_usage.to_csv("outputs/monthly_usage_count.csv", index=False)

# Close the connection
conn.close()

Monthly Customer Adoption:
Empty DataFrame
Columns: [transaction_month, transaction_type, new_adopters]
Index: []

Monthly Usage Count:
Empty DataFrame
Columns: [month, active_account_count]
Index: []
